# GPS Mobility Features - Exploration & Validation

This notebook explores the GPS mobility features extracted in Phase 2.

## Features Extracted
Following Saeb et al. (2015):
1. **Location variance** (log-transformed) - strongest depression predictor
2. **Distance traveled** (mean/std)
3. **Radius of gyration** (spatial spread)
4. **Home stay ratio** (social withdrawal proxy)
5. **Location diversity** (number of significant locations via DBSCAN)
6. **Movement entropy** (predictability of hourly patterns)
7. **Variability metrics** (CV for distance and radius)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Load Features and Labels

In [ ]:
# Load GPS features
gps_features = pd.read_parquet('../data/processed/features/gps_mobility_features.parquet')

# Load PHQ-9 labels
item9_labels = pd.read_csv('../data/processed/labels/item9_labels_pre.csv')
phq9_labels = pd.read_csv('../data/processed/labels/phq9_labels_pre.csv')

# Merge with labels
df = gps_features.merge(item9_labels, on='uid', how='left')
df = df.merge(phq9_labels[['uid', 'phq9_total', 'severity_category']], on='uid', how='left')

print(f"Total users with GPS features: {len(df)}")
print(f"Users with suicidal ideation: {df['item9_binary'].sum()}")
print(f"\nClass distribution:")
print(df['item9_binary'].value_counts())

## Feature Summary Statistics

In [ ]:
# Select feature columns (exclude uid and metadata)
feature_cols = [
    'location_variance_mean', 'location_variance_std',
    'distance_traveled_mean', 'distance_traveled_std', 'distance_traveled_cv',
    'radius_of_gyration_mean', 'radius_gyration_cv',
    'max_distance_from_home', 'home_stay_ratio',
    'n_significant_locations', 'movement_entropy'
]

# Summary statistics
df[feature_cols].describe().T

## Data Quality Check

In [ ]:
# Check for missing values
print("Missing values per feature:")
print(df[feature_cols].isnull().sum())

# Check data coverage
print("\nGPS data coverage:")
print(f"Mean valid days: {df['gps_valid_days'].mean():.1f} ± {df['gps_valid_days'].std():.1f}")
print(f"Min: {df['gps_valid_days'].min():.0f}, Max: {df['gps_valid_days'].max():.0f}")
print(f"Mean points per day: {df['gps_points_per_day'].mean():.1f}")

## Feature Distributions by Outcome

In [ ]:
# Key features to visualize
key_features = [
    'location_variance_mean',
    'distance_traveled_mean',
    'home_stay_ratio',
    'n_significant_locations'
]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

for i, feature in enumerate(key_features):
    ax = axes[i]
    
    # Violin plot by outcome
    sns.violinplot(data=df, x='item9_binary', y=feature, ax=ax)
    ax.set_xlabel('Suicidal Ideation (0=No, 1=Yes)')
    ax.set_title(f'{feature}\nby PHQ-9 Item #9')
    
    # Add sample sizes
    n0 = (df['item9_binary'] == 0).sum()
    n1 = (df['item9_binary'] == 1).sum()
    ax.text(0, ax.get_ylim()[1], f'n={n0}', ha='center', va='bottom')
    ax.text(1, ax.get_ylim()[1], f'n={n1}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## Correlation Analysis

In [ ]:
# Compute correlation matrix
corr_matrix = df[feature_cols].corr()

# Plot heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1)
plt.title('GPS Feature Correlation Matrix')
plt.tight_layout()
plt.show()

# Identify highly correlated pairs (|r| > 0.9)
high_corr = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > 0.9:
            high_corr.append((
                corr_matrix.columns[i],
                corr_matrix.columns[j],
                corr_matrix.iloc[i, j]
            ))

if high_corr:
    print("\nHighly correlated features (|r| > 0.9):")
    for f1, f2, r in high_corr:
        print(f"  {f1} <-> {f2}: r={r:.3f}")
else:
    print("\nNo highly correlated feature pairs (|r| > 0.9)")

## Univariate Association with Outcome

Test association between each GPS feature and suicidal ideation.

In [ ]:
from scipy.stats import mannwhitneyu

# Perform Mann-Whitney U test for each feature
results = []

for feature in feature_cols:
    # Split by outcome
    group0 = df[df['item9_binary'] == 0][feature].dropna()
    group1 = df[df['item9_binary'] == 1][feature].dropna()
    
    # Mann-Whitney U test (non-parametric)
    if len(group0) > 0 and len(group1) > 0:
        stat, p_value = mannwhitneyu(group0, group1, alternative='two-sided')
        
        results.append({
            'feature': feature,
            'mean_no_ideation': group0.mean(),
            'mean_ideation': group1.mean(),
            'difference': group1.mean() - group0.mean(),
            'p_value': p_value
        })

# Create results dataframe
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('p_value')

print("\nUnivariate associations with suicidal ideation (Mann-Whitney U test):")
print("=" * 80)
for _, row in results_df.iterrows():
    sig = "***" if row['p_value'] < 0.001 else "**" if row['p_value'] < 0.01 else "*" if row['p_value'] < 0.05 else ""
    print(f"{row['feature']:30s} p={row['p_value']:.4f} {sig:3s} | diff={row['difference']:7.3f}")

print("\nNote: Small sample size (n=4 with ideation) limits statistical power.")

## Key Findings Summary

Based on the analysis above, document:
1. Features with strongest association to outcome
2. Direction of effects (protective vs. risk)
3. Data quality issues
4. Recommendations for feature selection

In [ ]:
# Summary statistics by outcome
print("GPS Features by Suicidal Ideation Status")
print("=" * 80)
print("\nNo ideation (n=42):")
print(df[df['item9_binary'] == 0][feature_cols].describe().loc[['mean', 'std']].T)
print("\nWith ideation (n=4):")
print(df[df['item9_binary'] == 1][feature_cols].describe().loc[['mean', 'std']].T)